In [ ]:
# Artificial Neural Network

# Installing Theano
# pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras

# Part 1 - Data Preprocessing

In [1]:
!pip install -r requirements.txt

    100% |████████████████████████████████| 92.5MB 120kB/s ta 0:00:011 0% |▎                               | 819kB 1.7MB/s eta 0:00:54    3% |█▏                              | 3.2MB 1.9MB/s eta 0:00:48    5% |█▉                              | 5.3MB 1.9MB/s eta 0:00:46    6% |██                              | 5.6MB 2.1MB/s eta 0:00:41    10% |███▎                            | 9.6MB 776kB/s eta 0:01:47    11% |███▉                            | 11.0MB 1.4MB/s eta 0:00:58    13% |████▌                           | 12.9MB 2.0MB/s eta 0:00:40    14% |████▋                           | 13.3MB 1.7MB/s eta 0:00:48    15% |█████                           | 14.4MB 1.7MB/s eta 0:00:47    17% |█████▌                          | 15.9MB 2.2MB/s eta 0:00:35    20% |██████▊                         | 19.3MB 4.4MB/s eta 0:00:17    21% |██████▊                         | 19.5MB 1.4MB/s eta 0:00:51    22% |███████▏                        | 20.8MB 1.7MB/s eta 0:00:43    22% |███████▎                        | 21

    100% |████████████████████████████████| 10.0MB 220kB/s ta 0:00:01  1% |▍                               | 112kB 1.0MB/s eta 0:00:10    5% |█▊                              | 532kB 1.7MB/s eta 0:00:06    9% |███                             | 931kB 1.7MB/s eta 0:00:06    12% |████                            | 1.2MB 1.7MB/s eta 0:00:06    30% |█████████▊                      | 3.1MB 1.4MB/s eta 0:00:05    46% |██████████████▉                 | 4.6MB 799kB/s eta 0:00:07    78% |█████████████████████████       | 7.9MB 1.7MB/s eta 0:00:02    88% |████████████████████████████▍   | 8.9MB 2.1MB/s eta 0:00:01
    100% |████████████████████████████████| 13.0MB 1.5MB/s ta 0:00:01  3% |█▎                              | 501kB 2.1MB/s eta 0:00:06    10% |███▎                            | 1.3MB 1.7MB/s eta 0:00:07    11% |███▌                            | 1.4MB 1.8MB/s eta 0:00:07    18% |██████                          | 2.4MB 1.4MB/s eta 0:00:08    29% |█████████▍                      | 3.8MB 1.8M

    100% |████████████████████████████████| 92kB 2.4MB/s ta 0:00:011
    100% |████████████████████████████████| 337kB 1.5MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 2.1MB/s ta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
  Stored in directory: /root/.cache/pip/wheels/ee/98/38/46cbcc5a93cfea5492d19c38562691ddb23b940176c14f7b48
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built sklearn absl-py termcolor gast pyyaml
  Found existing installation: numpy 1.15.2
    Uninstalling numpy-1.15.2:
      Successfully uninstalled numpy-1.15.2
  Found exist

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [4]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

/usr/local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [57]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [58]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 3s 382us/step - loss: 0.5192 - acc: 0.7951
Epoch 2/100
8000/8000 [==============================] - 2s 203us/step - loss: 0.4315 - acc: 0.7977
Epoch 3/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4293 - acc: 0.8030
Epoch 4/100
8000/8000 [==============================] - 2s 204us/step - loss: 0.4285 - acc: 0.8012
Epoch 5/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4278 - acc: 0.8012
Epoch 6/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.4259 - acc: 0.7991
Epoch 7/100
8000/8000 [==============================] - 2s 205us/step - loss: 0.4220 - acc: 0.8017
Epoch 8/100
8000/8000 [==============================] - 2s 210us/step - loss: 0.4152 - acc: 0.8052
Epoch 9/100
8000/8000 [==============================] - 2s 236us/step - loss: 0.4045 - acc: 0.8120
Epoch 10/100
8000/8000 [==============================] - 2s 219us/step - loss: 0.3932 - acc: 0.8207

8000/8000 [==============================] - 2s 278us/step - loss: 0.3423 - acc: 0.8604
Epoch 82/100
8000/8000 [==============================] - 2s 248us/step - loss: 0.3425 - acc: 0.8595
Epoch 83/100
8000/8000 [==============================] - 2s 251us/step - loss: 0.3424 - acc: 0.8602
Epoch 84/100
8000/8000 [==============================] - 6s 729us/step - loss: 0.3423 - acc: 0.8596
Epoch 85/100
8000/8000 [==============================] - 3s 393us/step - loss: 0.3425 - acc: 0.8592
Epoch 86/100
8000/8000 [==============================] - 2s 209us/step - loss: 0.3417 - acc: 0.8610 2s - l
Epoch 87/100
8000/8000 [==============================] - 1s 172us/step - loss: 0.3424 - acc: 0.8605
Epoch 88/100
8000/8000 [==============================] - 1s 167us/step - loss: 0.3422 - acc: 0.8595
Epoch 89/100
8000/8000 [==============================] - 2s 247us/step - loss: 0.3420 - acc: 0.8620
Epoch 90/100
8000/8000 [==============================] - 2s 196us/step - loss: 0.3423 - acc: 0.8

In [53]:

# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[1508   87]
 [ 189  216]]


In [54]:
# relu relu sigmoid
# loss: 0.3993 - acc: 0.8364
# array([[1537,   58],
#        [ 253,  152]])


# relu relu relu
# loss: 0.3509 - acc: 0.8534
# [[1508   87]
#  [ 189  216]]

# softmax relu tanh
# loss: 0.3335 - acc: 0.8650
# array([[1501,   94],
#       [ 192,  213]])

# relu softmax tanh
# loss: 0.3327 - acc: 0.8665
# array([[1534,   61],
#        [ 206,  199]])

# relu relu tanh
# loss: 0.3374 - acc: 0.8582
# array([[1529,   66],
#        [ 202,  203]])